El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [1]:
#importamos las librerias principales a ocupar.
import pandas as pd #
import numpy as np #
import time #
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder #
from sklearn.linear_model import LinearRegression #
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error #
import lightgbm as lgb


In [2]:
raw_df = pd.read_csv('/datasets/car_data.csv')
raw_df.info()
print(raw_df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [3]:
#quitaria columnas que siento que no son importantes para el analisis.
#Date Crawled, Date Created, NumberOfPictures, PostalCode, LastSeen
#tambien quitaria los que tienen precio 0 ya que me pueden dar numeros muy bajos al momento de 
#crear los modelos
df_clean = raw_df.copy()
cols_to_drop = ['DateCrawled','DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen']
cols_to_drop = [c for c in cols_to_drop if c in df_clean.columns]
df_clean = df_clean.drop(columns=cols_to_drop)
print(df_clean.head(10))
df_clean.info()
df_clean = df_clean[df_clean['Price'] > 0]
df_clean = df_clean.dropna(subset=['Price'])
print(df_clean.head(10))

   Price  VehicleType  RegistrationYear Gearbox  Power    Model  Mileage  \
0    480          NaN              1993  manual      0     golf   150000   
1  18300        coupe              2011  manual    190      NaN   125000   
2   9800          suv              2004    auto    163    grand   125000   
3   1500        small              2001  manual     75     golf   150000   
4   3600        small              2008  manual     69    fabia    90000   
5    650        sedan              1995  manual    102      3er   150000   
6   2200  convertible              2004  manual    109  2_reihe   150000   
7      0        sedan              1980  manual     50    other    40000   
8  14500          bus              2014  manual    125    c_max    30000   
9    999        small              1998  manual    101     golf   150000   

   RegistrationMonth  FuelType       Brand NotRepaired  
0                  0    petrol  volkswagen         NaN  
1                  5  gasoline        audi       

In [4]:
df_clean.info()
df_clean.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 343597 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              343597 non-null  int64 
 1   VehicleType        309847 non-null  object
 2   RegistrationYear   343597 non-null  int64 
 3   Gearbox            326242 non-null  object
 4   Power              343597 non-null  int64 
 5   Model              326076 non-null  object
 6   Mileage            343597 non-null  int64 
 7   RegistrationMonth  343597 non-null  int64 
 8   FuelType           314138 non-null  object
 9   Brand              343597 non-null  object
 10  NotRepaired        277731 non-null  object
dtypes: int64(5), object(6)
memory usage: 31.5+ MB


,Price,RegistrationYear,Power,Mileage,RegistrationMonth
count,343597.000000,343597.000000,343597.000000,343597.000000,343597.000000
mean,4555.121974,2004.089797,111.000192,128337.616452,5.777891
std,4515.058553,78.413225,188.177974,37521.047429,3.698186
min,1.000000,1000.000000,0.000000,5000.000000,0.000000
25%,1200.000000,1999.000000,69.000000,125000.000000,3.000000
50%,2890.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6500.000000,2008.000000,143.000000,150000.000000,9.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000


In [5]:
#hay 2 cosas que me preocupan el precio que el min es de 1. y el year que es de 1000 y 9999.
print(df_clean['RegistrationYear'].value_counts().sort_index())
print(df_clean[df_clean['RegistrationYear'] == 1000])
#despues de imprimir para revisar los del year 1000. opto por solo mantener los datos del 1950 al 2016
#y volver a revisar los detalles del df.

1000    31
1001     1
1039     1
1111     1
1234     4
        ..
8500     1
8888     1
9000     2
9450     1
9999    18
Name: RegistrationYear, Length: 140, dtype: int64
        Price VehicleType  RegistrationYear Gearbox  Power     Model  Mileage  \
16062     190         NaN              1000     NaN      0    mondeo     5000   
53577     330         NaN              1000     NaN      0      polo     5000   
55605     500         NaN              1000     NaN      0       NaN     5000   
60017      80         NaN              1000     NaN      0       NaN     5000   
66198    1300         NaN              1000    auto      0       NaN     5000   
71459     500         NaN              1000     NaN      0      golf     5000   
79120    4750         NaN              1000     NaN      0     other     5000   
91869     400         NaN              1000     NaN      0       NaN     5000   
97736     140         NaN              1000     NaN      0       911     5000   
110123    400      

In [6]:
df_clean = df_clean[(df_clean['RegistrationYear'] >= 1950) & (df_clean['RegistrationYear'] <= 2016)]
print(df_clean.head(10))
df_clean.info()
df_clean.describe()

    Price  VehicleType  RegistrationYear Gearbox  Power    Model  Mileage  \
0     480          NaN              1993  manual      0     golf   150000   
1   18300        coupe              2011  manual    190      NaN   125000   
2    9800          suv              2004    auto    163    grand   125000   
3    1500        small              2001  manual     75     golf   150000   
4    3600        small              2008  manual     69    fabia    90000   
5     650        sedan              1995  manual    102      3er   150000   
6    2200  convertible              2004  manual    109  2_reihe   150000   
8   14500          bus              2014  manual    125    c_max    30000   
9     999        small              1998  manual    101     golf   150000   
10   2000        sedan              2004  manual    105  3_reihe   150000   

    RegistrationMonth  FuelType       Brand NotRepaired  
0                   0    petrol  volkswagen         NaN  
1                   5  gasoline     

,Price,RegistrationYear,Power,Mileage,RegistrationMonth
count,329639.000000,329639.000000,329639.000000,329639.000000,329639.000000
mean,4606.592615,2002.623303,111.932344,128232.414854,5.820616
std,4545.446365,6.754713,184.861470,37487.911156,3.678957
min,1.000000,1950.000000,0.000000,5000.000000,0.000000
25%,1200.000000,1999.000000,71.000000,125000.000000,3.000000
50%,2900.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6600.000000,2007.000000,143.000000,150000.000000,9.000000
max,20000.000000,2016.000000,20000.000000,150000.000000,12.000000


In [7]:
print(df_clean['Price'].value_counts().sort_index().head(50))
#despues de revisar esto busque la forma de poder hacer un recorte para no afectar mis modelos de 
#entrenamiento mas adelante opte por buscar alguna forma de quitar valores muy bajos al mismo tiempo
#valores muy altos. 
df_clean = df_clean[(df_clean['Price'] > df_clean['Price'].quantile(0.01)) & 
    (df_clean['Price'] < df_clean['Price'].quantile(0.99))]
print(df_clean.head(10))
df_clean.info()
df_clean.describe()

1     1106
2       11
3        7
4        1
5       26
7        3
8        9
9        8
10      83
11       5
12       8
13       7
14       5
15      25
16       2
17       5
18       2
19       3
20      47
21       1
24       1
25      31
26       1
27       1
29       2
30      52
32       1
33       1
35      18
38       1
39       6
40      45
45      15
47       1
49      11
50     318
55      18
58       1
59       5
60      56
65      18
66       2
69       3
70      69
74       1
75      53
77       1
79       4
80     143
85      10
Name: Price, dtype: int64
    Price  VehicleType  RegistrationYear Gearbox  Power    Model  Mileage  \
0     480          NaN              1993  manual      0     golf   150000   
1   18300        coupe              2011  manual    190      NaN   125000   
2    9800          suv              2004    auto    163    grand   125000   
3    1500        small              2001  manual     75     golf   150000   
4    3600        small              200

,Price,RegistrationYear,Power,Mileage,RegistrationMonth
count,322875.000000,322875.000000,322875.000000,322875.000000,322875.000000
mean,4500.921316,2002.601874,111.757869,128801.626016,5.844153
std,4308.282810,6.674716,186.022326,36853.904887,3.669147
min,101.000000,1950.000000,0.000000,5000.000000,0.000000
25%,1200.000000,1999.000000,71.000000,125000.000000,3.000000
50%,2900.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6500.000000,2007.000000,143.000000,150000.000000,9.000000
max,18899.000000,2016.000000,20000.000000,150000.000000,12.000000


In [8]:
print(df_clean.isnull().sum())

Price                    0
VehicleType          18484
RegistrationYear         0
Gearbox              14290
Power                    0
Model                14854
Mileage                  0
RegistrationMonth        0
FuelType             22659
Brand                    0
NotRepaired          57692
dtype: int64


In [9]:
print("\nDistribución de VehicleType:")
print(df_clean['VehicleType'].value_counts())
print("\nDistribución de Gearbox:")
print(df_clean['Gearbox'].value_counts())
print("\nDistribución de Model:")
print(df_clean['Model'].value_counts())
print("\nDistribución de Fueltype:")
print(df_clean['FuelType'].value_counts())
print("\nDistribución de Notrepaired:")
print(df_clean['NotRepaired'].value_counts())


Distribución de VehicleType:
sedan          88035
small          76923
wagon          62792
bus            27930
convertible    19316
coupe          15095
suv            11289
other           3011
Name: VehicleType, dtype: int64

Distribución de Gearbox:
manual    247541
auto       61044
Name: Gearbox, dtype: int64

Distribución de Model:
golf                  26277
other                 22580
3er                   18362
polo                  11814
corsa                 11334
                      ...  
gl                        4
serie_3                   3
rangerover                3
range_rover_evoque        2
serie_1                   1
Name: Model, Length: 250, dtype: int64

Distribución de Fueltype:
petrol      201433
gasoline     92821
lpg           4991
cng            529
hybrid         211
other          148
electric        83
Name: FuelType, dtype: int64

Distribución de Notrepaired:
no     233216
yes     31967
Name: NotRepaired, dtype: int64


In [10]:
# Rellenar valores faltantes con 'unknown'
categorical_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']

for col in categorical_columns:
    df_clean[col] = df_clean[col].fillna('unknown')

# Verificar que no quedan valores faltantes
print(df_clean.isnull().sum())

Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Mileage              0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
dtype: int64


hasta aqui termina mi limpieza del DF. Los datos fueron limpiados y ya no tenemos datos faltantes.

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Muy buen trabajo con el tratamiento y análisis de los datos, siempre en un proyecto lo importante es primero entender los datos con los que se trabajará antes de pasar al modelado
</div>

## Entrenamiento del modelo 

In [11]:
#para empezar a entrenar necesitamos 1 columna mas. que tan viejo es el carro.
df_clean['CarAge'] = 2016 - df_clean['RegistrationYear']
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322875 entries, 0 to 354368
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              322875 non-null  int64 
 1   VehicleType        322875 non-null  object
 2   RegistrationYear   322875 non-null  int64 
 3   Gearbox            322875 non-null  object
 4   Power              322875 non-null  int64 
 5   Model              322875 non-null  object
 6   Mileage            322875 non-null  int64 
 7   RegistrationMonth  322875 non-null  int64 
 8   FuelType           322875 non-null  object
 9   Brand              322875 non-null  object
 10  NotRepaired        322875 non-null  object
 11  CarAge             322875 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 32.0+ MB


In [12]:
# Separar features y target
X = df_clean.drop('Price', axis=1)
y = df_clean['Price']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=12345)

print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")

Train shape: (258300, 11)
Test shape: (64575, 11)


In [13]:
categorical_cols = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
numerical_cols = ['RegistrationYear', 'Power', 'Mileage', 'RegistrationMonth', 'CarAge']

# Función para RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [14]:
# Para regresión lineal necesitamos One-Hot Encoding
X_train_lr = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_test_lr = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)

# Asegurar que train y test tengan las mismas columnas
X_test_lr = X_test_lr.reindex(columns=X_train_lr.columns, fill_value=0)

print(f"Shape después de One-Hot: {X_train_lr.shape}")

Shape después de One-Hot: (258300, 312)


In [15]:
%%time
# Entrenar modelo
lr_model = LinearRegression()
start_time = time.time()
lr_model.fit(X_train_lr, y_train)
lr_training_time = time.time() - start_time
# Predicciones
start_time = time.time()
y_pred_lr = lr_model.predict(X_test_lr)
lr_prediction_time = time.time() - start_time
# Evaluar
rmse_lr = rmse(y_test, y_pred_lr)
print(f"RMSE Regresión Lineal: {rmse_lr:.2f}")
print(f"Training Time Regresión Lineal: {lr_training_time:.2f}")
print(f"Prediction Time Regresión Lineal: {lr_prediction_time:.2f}")

RMSE Regresión Lineal: 2684.97
Training Time Regresión Lineal: 8.55
Prediction Time Regresión Lineal: 0.19
CPU times: user 12 s, sys: 4.55 s, total: 16.5 s
Wall time: 8.74 s


In [16]:
# Preparar datos para Random Forest (similar a Linear Regression)
X_train_rf = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_test_rf = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)

# Asegurar que train y test tengan las mismas columnas
X_test_rf = X_test_rf.reindex(columns=X_train_rf.columns, fill_value=0)

print(f"Shape para Random Forest: {X_train_rf.shape}")

Shape para Random Forest: (258300, 312)


In [17]:
%%time
# Entrenar Random Forest
rf_model = RandomForestRegressor(
    n_estimators=100,  # Número de árboles
    random_state=12345,
    n_jobs=-1  # Usar todos los cores
)
start_time = time.time()
rf_model.fit(X_train_rf, y_train)
rf_training_time = time.time() - start_time

start_time = time.time()
y_pred_rf = rf_model.predict(X_test_rf)
rf_prediction_time = time.time() - start_time

rmse_rf = rmse(y_test, y_pred_rf)
print(f"RMSE Random Forest: {rmse_rf:.2f}")
print(f"Training Time Random Forest: {rf_training_time:.2f}")
print(f"Prediction Time Random Forest: {rf_prediction_time:.2f}")

RMSE Random Forest: 1521.41
Training Time Random Forest: 189.02
Prediction Time Random Forest: 1.62
CPU times: user 6min 15s, sys: 1.01 s, total: 6min 16s
Wall time: 3min 10s


<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Correcto, se está calculando el tiempo de entrenamiento y predicción por separado lo cual dará información necesaria para cuando se quiera llevar los modelos a un entorno productivo
</div>

In [18]:
# Preparar datos para LightGBM
# LightGBM puede manejar variables categóricas directamente
X_train_lgb = X_train.copy()
X_test_lgb = X_test.copy()

# Convertir variables categóricas a tipo 'category' para LightGBM
for col in categorical_cols:
    X_train_lgb[col] = X_train_lgb[col].astype('category')
    X_test_lgb[col] = X_test_lgb[col].astype('category')

print(f"Shape para LightGBM: {X_train_lgb.shape}")
print("Tipos de datos:")
print(X_train_lgb.dtypes)

Shape para LightGBM: (258300, 11)
Tipos de datos:
VehicleType          category
RegistrationYear        int64
Gearbox              category
Power                   int64
Model                category
Mileage                 int64
RegistrationMonth       int64
FuelType             category
Brand                category
NotRepaired          category
CarAge                  int64
dtype: object


In [19]:
# Definir el espacio de hiperparámetros para LightGBM
param_distributions = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10, -1],  # -1 significa sin límite
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'num_leaves': [31, 50, 100, 200],
    'min_child_samples': [20, 30, 50],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

print("Hiperparámetros a optimizar:")
for param, values in param_distributions.items():
    print(f"- {param}: {values}")

Hiperparámetros a optimizar:
- n_estimators: [100, 200, 300]
- max_depth: [3, 5, 7, 10, -1]
- learning_rate: [0.01, 0.05, 0.1, 0.2]
- num_leaves: [31, 50, 100, 200]
- min_child_samples: [20, 30, 50]
- subsample: [0.8, 0.9, 1.0]
- colsample_bytree: [0.8, 0.9, 1.0]


In [20]:
%%time
# Configurar RandomizedSearchCV para LightGBM
lgb_model = lgb.LGBMRegressor(
    random_state=12345,
    n_jobs=-1,
    verbose=-1  # Silenciar warnings
)

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=lgb_model,
    param_distributions=param_distributions,
    n_iter=100,  # 100 combinaciones aleatorias
    cv=3,        # 3-fold cross validation
    scoring='neg_mean_squared_error',
    random_state=12345,
    n_jobs=-1,
    verbose=1    # Mostrar progreso
)

print("Iniciando búsqueda de hiperparámetros...")
print("Esto puede tomar varios minutos...")

# Entrenar con búsqueda de hiperparámetros
start_time = time.time()
random_search.fit(X_train_lgb, y_train)
lgb_lookup_time = time.time() - start_time

print(f"\n¡Búsqueda completada!")
print(f"Mejor score (CV): {-random_search.best_score_:.2f}")
print(f"Lookup Time LGBMRegressor : {lgb_lookup_time:.2f}")
print(f"Mejores hiperparámetros:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

Iniciando búsqueda de hiperparámetros...
Esto puede tomar varios minutos...
Fitting 3 folds for each of 100 candidates, totalling 300 fits

¡Búsqueda completada!
Mejor score (CV): 2133106.42
Lookup Time LGBMRegressor : 488.04
Mejores hiperparámetros:
  subsample: 0.8
  num_leaves: 200
  n_estimators: 300
  min_child_samples: 30
  max_depth: -1
  learning_rate: 0.1
  colsample_bytree: 0.8
CPU times: user 28.4 s, sys: 962 ms, total: 29.4 s
Wall time: 8min 8s


In [21]:
# El score real es:
mse = 2133106.42
rmse_cv = np.sqrt(mse)
print(f"RMSE en validación cruzada: {rmse_cv:.2f}")

RMSE en validación cruzada: 1460.52


In [22]:
%%time
# Entrenar el modelo final con los mejores hiperparámetros
best_lgb_model = lgb.LGBMRegressor(
    **random_search.best_params_,
    random_state=12345,
    n_jobs=-1,
    verbose=-1
)

# Entrenar con todos los datos de entrenamiento
start_time = time.time()
best_lgb_model.fit(X_train_lgb, y_train)
lgb_training_time = time.time() - start_time

# Hacer predicciones en el conjunto de prueba
start_time = time.time()
y_pred_lgb = best_lgb_model.predict(X_test_lgb)
lgb_prediction_time = time.time() - start_time

# Calcular RMSE en el conjunto de prueba
rmse_lgb = rmse(y_test, y_pred_lgb)

print(f"RESULTADOS FINALES:")
print(f"RMSE LightGBM (validación cruzada): {rmse_cv:.2f}")
print(f"RMSE LightGBM (conjunto de prueba): {rmse_lgb:.2f}")
print(f"Training Time LGBMRegressor: {lgb_training_time:.2f}")
print(f"Prediction Time LGBMRegressor: {lgb_prediction_time:.2f}")


RESULTADOS FINALES:
RMSE LightGBM (validación cruzada): 1460.52
RMSE LightGBM (conjunto de prueba): 1454.74
Training Time LGBMRegressor: 12.71
Prediction Time LGBMRegressor: 1.22
CPU times: user 27.8 s, sys: 123 ms, total: 27.9 s
Wall time: 13.9 s


In [23]:
!pip install catboost

In [24]:
%%time
import catboost as cb
from catboost import CatBoostRegressor

# Preparar datos para CatBoost
# CatBoost maneja automáticamente las variables categóricas
X_train_cb = X_train.copy()
X_test_cb = X_test.copy()

# Especificar qué columnas son categóricas
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

print("Entrenando CatBoost...")
print("Configuración inicial con parámetros por defecto")

# Crear y entrenar el modelo CatBoost
cb_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    cat_features=cat_features,
    random_seed=12345,
    verbose=100  # Mostrar progreso cada 100 iteraciones
)

# Entrenar el modelo
start_time = time.time()
cb_model.fit(X_train_cb, y_train)
cb_training_time = time.time() - start_time
# Hacer predicciones
start_time = time.time()
y_pred_cb = cb_model.predict(X_test_cb)
cb_prediction_time = time.time() - start_time
# Calcular RMSE
rmse_cb = rmse(y_test, y_pred_cb)

print(f"\nRESULTADOS CatBoost:")
print(f"RMSE CatBoost: {rmse_cb:.2f}")
print(f"Training Time CatBoost: {cb_training_time:.2f}")
print(f"Prediction Time CatBoost: {cb_prediction_time:.2f}")

Entrenando CatBoost...
Configuración inicial con parámetros por defecto
0:	learn: 4035.6614029	total: 217ms	remaining: 3m 36s
100:	learn: 1693.3486201	total: 11.1s	remaining: 1m 38s
200:	learn: 1614.6466662	total: 21.9s	remaining: 1m 27s
300:	learn: 1573.5016858	total: 32.5s	remaining: 1m 15s
400:	learn: 1547.5721729	total: 42.8s	remaining: 1m 3s
500:	learn: 1529.2422712	total: 52.9s	remaining: 52.7s
600:	learn: 1514.0627125	total: 1m 3s	remaining: 42s
700:	learn: 1501.6768391	total: 1m 14s	remaining: 31.6s
800:	learn: 1490.5989496	total: 1m 25s	remaining: 21.1s
900:	learn: 1482.5104065	total: 1m 35s	remaining: 10.5s
999:	learn: 1473.6347411	total: 1m 46s	remaining: 0us

RESULTADOS CatBoost:
RMSE CatBoost: 1544.74
Training Time CatBoost: 107.29
Prediction Time CatBoost: 0.21
CPU times: user 3min 29s, sys: 1.44 s, total: 3min 30s
Wall time: 1min 47s


## Análisis del modelo

In [25]:
# Crear diccionario para almacenar resultados
df_results = pd.DataFrame({
    'Modelo' : ['Linear Regression', 'Random Forest', 'LightGBM', 'CatBoost'],
    'RMSE' : [rmse_lr, rmse_rf, rmse_lgb, rmse_cb],
    'Tiempo_Entrenamiento (s)' : [lr_training_time, rf_training_time, lgb_training_time, cb_training_time],
    'Tiempo_Prediccion (s)' : [lr_prediction_time, rf_prediction_time, lgb_prediction_time, cb_prediction_time]
})

In [26]:

print(df_results)

              Modelo         RMSE  Tiempo_Entrenamiento (s)  \
0  Linear Regression  2684.968248                  8.548034   
1      Random Forest  1521.412895                189.023903   
2           LightGBM  1454.740224                 12.712402   
3           CatBoost  1544.737479                107.294132   

   Tiempo_Prediccion (s)  
0               0.188567  
1               1.624249  
2               1.221132  
3               0.206965  


Conclusion:
Tras entrenar y evaluar los diferentes modelos de prediccion (Regresion Lineal, Bosque Aleatorio, LightGBM y CatBoost), se observa que LightGBM presenta el menor error (RMSE = 1454.74), lo que indica una mayor precision en las predicciones, su tiempo de entrenamiento (≈ 12.7 s) es inferior al de otros modelos y comparable con el de la Regresión Lineal, lo que lo convierte en la mejor opcion.

Manteniendo los hiperparametros optimos obtenidos mediante RandomizedSearchCV, el modelo LightGBM es el que yo le recomendaria a Rusty Bargain, ya que ofrece un equilibrio entre precision y velocidad.

<div class="alert alert-block alert-success">
<b>Comentario general (1ra Iteracion)</b> <a class=“tocSkip”></a>

Muy bien, se evaluaron correctamente las métricas solicitadas, en cuanto a las métricas de desempeño de las predicciones cómo en el tiempo que toma cada una de sus etapas. El medir el tiempo es importante ya que cuando pones los modelos en producción para que puedan ser usados se suele realizar por medio de API donde se prioriza más el tiempo de predicción que la precisión cómo tal. Lo ideal es buscar siempre el balance entre buen desempeño de predicciones y de tiempo de predicción para tu modelo, sobre todo cuando va a ser usado en tiempo real.
    
Saludos!
</div>

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos